In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm

# Scenario: Creating a Set of Machine Learning Friendly Features from EHR Data to Predict Diabetes Onset

First we will load in the necessary data files

In [ ]:
def load_data_for_file(filename):
    print(f"Loading data for {filename}")
    df = pd.concat([ # use pd.concat to append/concatenate the data for all states together into a single frame
        pd.read_parquet(f"https://dicbworkshops.s3.amazonaws.com/{output_dir}/parquet/{filename}") # use read_csv to load the data from each output directory
        for output_dir in tqdm(['output_hi_small', 'output_ma_small', 'output_tx_small', 'output_wa_small']) # loop over each output directory
    ])
    return df

In [ ]:
# load in the conditions
conditions = load_data_for_file('conditions.parquet')
observations = load_data_for_file('observations.parquet')
medications = load_data_for_file('medications.parquet')
procedures = load_data_for_file('procedures.parquet')

## Filtering Out Patients with Diagnoses of Type-2 Diabetes
For this exercise, we are interested in filtering out all patients with a diagnosis of Type-2 diabetes \
We can filter these out based on the SNOMED code `44054006`

In [ ]:
type2_patients = conditions.query('CODE == 44054006').sort_values(by=['PATIENT', 'START']).drop_duplicates(subset=['PATIENT', 'START'], keep='first')

In [ ]:
# now we create a dictionary/lookup table to map each patient's ID to the date of their earliest Type 2 diagnosis
patient_diagnosis_dates = {
    row['PATIENT']: row['START']
    for _, row in type2_patients.iterrows()
}

In [ ]:
conditions

In [ ]:
# add simplified date columns to the observations, medications, and procedures 
observations_simplified = observations.assign(
    DATE_SIMPLE=lambda x: pd.to_datetime(x['DATE']).dt.date.astype('str')
)
medications_simplified = medications.assign(
    DATE_SIMPLE=lambda x: pd.to_datetime(x['START']).dt.date.astype('str')
)
procedures_simplified = procedures.assign(
    DATE_SIMPLE=lambda x: pd.to_datetime(x['START']).dt.date.astype('str')
)

## Filtering Out Post-diagnosis Conditions, Observations, Medications, and Procedures and Unifying Into a Shared Representation
First we need to filter out all EHR data from encounters that took place after the Type-2 diabetes diagnosis for the type 2 patients

In [ ]:
def filter_data(df, patients, date_column='DATE_SIMPLE'):
    data_filtered = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        patient = row['PATIENT']
        date = row[date_column]
        if patient in patients and patients[patient] > date:
            data_filtered.append(row)
    return pd.DataFrame(data_filtered)

In [ ]:
conditions_filtered = filter_data(conditions, patient_diagnosis_dates, 'START') 

In [ ]:
observations_subset = observations_simplified[observations_simplified['PATIENT'].isin(patient_diagnosis_dates)]
observations_filtered = filter_data(observations_subset, patient_diagnosis_dates)

In [ ]:
medications_subset = medications_simplified[medications_simplified['PATIENT'].isin(patient_diagnosis_dates)]
medications_filtered = filter_data(medications_subset, patient_diagnosis_dates)

In [ ]:
procedures_subset = procedures_simplified[procedures_simplified['PATIENT'].isin(patient_diagnosis_dates)]
procedures_filtered = filter_data(procedures_subset, patient_diagnosis_dates)


In [ ]:
procedures_filtered

Now we will label the EHR data for the type 2 and non-type 2 patients and unify everythin into a single event set

In [ ]:
def get_unified_records(conditions, observations, medications, procedures):
    return pd.concat([
    conditions[['PATIENT', 'START', 'CODE', 'DESCRIPTION']].assign(
        EVENT_TYPE='CONDITION',
    ).rename(columns={'START': 'DATE'}),
    observations[['PATIENT', 'DATE_SIMPLE', 'CODE', 'DESCRIPTION']].assign(
        EVENT_TYPE='OBSERVATION',
    ).rename(columns={'DATE_SIMPLE': 'DATE'}),
    medications[['PATIENT', 'DATE_SIMPLE', 'CODE', 'DESCRIPTION']].assign(
        EVENT_TYPE='MEDICATION',
    ).rename(columns={'DATE_SIMPLE': 'DATE'}),
    procedures[['PATIENT', 'DATE_SIMPLE', 'CODE', 'DESCRIPTION']].assign(
        EVENT_TYPE='PROCEDURE',
    ).rename(columns={'DATE_SIMPLE': 'DATE'})
    ])

In [ ]:
# starting with the Type 2 diabetes patients, we label and unify the data from medications, procedures, observations, and conditions into a single recordset
all_records_type2 = get_unified_records(conditions_filtered, observations_filtered, medications_filtered, procedures_filtered)

In [ ]:
# now we will do the same for the non type2 patients
all_records_non_type2 = get_unified_records(
    conditions[~conditions['PATIENT'].isin(type2_patients['PATIENT'])],
    observations_simplified[~observations_simplified['PATIENT'].isin(type2_patients['PATIENT'])],
    medications_simplified[~medications_simplified['PATIENT'].isin(type2_patients['PATIENT'])],
    procedures_simplified[~procedures_simplified['PATIENT'].isin(type2_patients['PATIENT'])],
)

In [ ]:
all_records_non_type2

## Now Like Good Machine Learning Practitioners, we will split our Type 2 and Non-Type 2 patients into Training and Hold-out sets


In [ ]:
from sklearn.model_selection import train_test_split
type2_patients = all_records_type2['PATIENT'].unique()
non_type2_patients = all_records_non_type2['PATIENT'].unique()
labels = np.concatenate([np.ones(type2_patients.shape), np.zeros(non_type2_patients.shape)])
train_patients, test_patients, train_labels, test_labels = train_test_split(np.concatenate([type2_patients, non_type2_patients]), labels, test_size=0.2, stratify=labels)

In [ ]:
# now split the records accordingly
all_records = pd.concat([all_records_type2, all_records_non_type2])
train_records = all_records[all_records['PATIENT'].isin(train_patients)]
test_records = all_records[all_records['PATIENT'].isin(test_patients)]

## Option 1: Bag of Labeled Clinical Encounters
The simplest feature representation we can create and test is a binary vector representation \
which encodes the occurence or lack-therof of different clinical encounters/events \
To construct this representation, we can use the scikit-learn package's `CountVectorizer` class

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(
    binary=True,
    tokenizer=lambda x: x.split('|'),
    token_pattern=None,
    lowercase=False
)

The vectorizer expects a `'|'` pipe delimited string of coded encounters, \
so we will construct this representation now for our training patients

In [ ]:
# condense the records into a pipe-delimited string of event tokens per patient, 
# where each token is of the form <EVENT_TYPE>::<CODE>
train_records_condensed = train_records.assign(
    EVENT_TOKEN=lambda x: x['EVENT_TYPE'] + '::' + x['CODE'].astype(str) + '|'
).groupby(['PATIENT'])['EVENT_TOKEN'].sum().reset_index()

In [ ]:
train_data_final = train_records_condensed.assign(
    LABEL=lambda x: x['PATIENT'].isin(type2_patients).astype(int)
)

In [ ]:
train_data_final

## Now that we have constructed a Pipe Delimited Event Representation, We Can Vectorize

In [ ]:
event_occurence_vectors = vectorizer.fit_transform(train_data_final['EVENT_TOKEN'])

In [ ]:
event_occurence_vectors.shape

## Now We Will Test the performance of this representation on the Training Set Using KFold Cross Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix

In [ ]:
kfold = StratifiedKFold(n_splits=5)
clf = RandomForestClassifier()

In [ ]:
metrics = []
for i, (train_index, test_index) in tqdm(enumerate(kfold.split(event_occurence_vectors, train_data_final['LABEL'])), total=5):
    train_x, train_y = event_occurence_vectors[train_index], train_data_final['LABEL'].to_numpy()[train_index]
    test_x, test_y = event_occurence_vectors[test_index], train_data_final['LABEL'].to_numpy()[test_index]
    # fit the model on the training fold
    clf.fit(train_x, train_y)
    # evaluate the model on the validation fold
    preds = clf.predict(test_x)
    scores = clf.predict_proba(test_x)[:, 1]
    # get the AUROC
    fpr, tpr, _ = roc_curve(test_y, scores)
    auroc = auc(fpr, tpr)
    # get the confusion matrix
    cm = confusion_matrix(test_y, preds)
    # save the metrics
    metrics.append({
        'AUROC': auroc,
        'Precision': cm[1, 1] / cm[:, 1].sum(),
        'Recall': cm[1, 1] / cm[1].sum(),
        'Specificity': cm[0, 0] / cm[0].sum()
    })
pd.DataFrame(metrics)
    

## The model appears to be performing well so far, but is that where the story ends, or does the plot thicken?
One thing that we can do is audit the way our current model is behaving, and what its predictions are based \
on, by looking at feature importance rankings. Here we inspect the top 50 important features

In [ ]:
reverse_lookup = {
    value: key for key, value in vectorizer.vocabulary_.items()
}
top_50_indices = np.argsort(clf.feature_importances_)[-50:]
top50_features = [reverse_lookup[idx] for idx in top_50_indices]
top50_importances = clf.feature_importances_[top_50_indices]
top_features_df = pd.DataFrame({
    'FEATURE_NAME': top50_features,
    'FEATURE_IMPORTANCE': top50_importances
}).assign(
    CODE=lambda x: x['FEATURE_NAME'].str.split('::').apply(lambda pair: pair[1])
).merge(
    all_records[['CODE', 'DESCRIPTION']].drop_duplicates().astype({'CODE': str}),
    on='CODE',
).sort_values(by='FEATURE_IMPORTANCE', ascending=False)

In [ ]:
top_features_df